# მონაცემთა მეცნიერება
##### მანქანური სწავლების გამოყენებამდე

### დუბლინის ხმაურის მონაცემები უბნების მიხედვით

საჯარო ინფორმაცია, გამოქვეყნებულია: https://data.smartdublin.ie/dataset/ambient-sound-monitoring-network


ცნობარი ხმაურის ერთეულების შესახებ: http://www.acoustic-glossary.co.uk/leq.htm

### რამდენიმე ბიბლიოთეკის ჩატვირთვა

In [ ]:
from __future__ import unicode_literals

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [ ]:
import glob

In [ ]:
%matplotlib notebook

ფაილებისა და ფოლდერების განლაგება

In [ ]:
%%bash
ls /home/revaz/data/public/dublin/raw/ballyfermot/2015/ | head

In [ ]:
%%bash
ls /home/revaz/data/public/dublin/raw/ballyfermot/2015/April\ 15/ | head

ფაილის ფორმატი: კომენტარი, გამყოფები, ინდექსი

In [ ]:
%%bash
head -n 15 '/home/revaz/data/public/dublin/raw/ballyfermot/2015/April 15/DCC-NOISE-001003-2015-04-08T00-00-00.txt'

ფაილის მისამართების სორტირებული სია

In [ ]:
# single district, one year time period
files = sorted(glob.glob('/home/revaz/data/public/dublin/raw/ballyfermot/2015/*/*.txt'))

In [ ]:
def get_header(file):
    """Get first 9 rows of each file as a list."""
    with open(files[0], 'r') as f:
        header = [f.readline() for _ in range(9)]
    
    return header

თანადობა თითოეული ფაილისა და მისი საწყისი კომენტარებისა

In [ ]:
headers = {file: get_header(file) for file in files} # dict comprehension

In [ ]:
# headers

აქ ვამოწმებთ ემთხვევა თუ არა კომენტარის შესაბამისი სტრიქონი ყველა ფაილში

In [ ]:
for i in range(9):
    unique_header_items = {v[i] for k, v in headers.items()} # set comprehension
    print(len(unique_header_items) == 1)
    # assert len(len(unique_header_items) == 1)


In [ ]:
len(files)

### ნედლი მონაცემების ჩატვირთვა

In [ ]:
all_dfs = [pd.read_csv(file, sep=',', index_col=0, parse_dates=True, dayfirst=True, skiprows=8)
           for file in files]

ერთი ფაილის ჩატვირთული ნიმუში

In [ ]:
all_dfs[130].head()

ფაილების გაერთიანება (კონკატენაცია)

In [ ]:
full_df = pd.concat(all_dfs)

In [ ]:
full_df.head()

საჭიროა ინდექსის სორტირება

In [ ]:
full_df.sort_index(inplace=True)

ინდექსისა და სვეტების სახელების გადარქმევა

In [ ]:
full_df.columns

In [ ]:
full_df.index.name = 'timestamp'

In [ ]:
full_df.rename(columns=lambda name: name.strip(), inplace=True)

### რაოდენობრივი ანალიზი

გამოტოვებული დროითი ინტერვალები

In [ ]:
time_index = full_df.index.to_series()

In [ ]:
time_diff = time_index - time_index.shift(1)

In [ ]:
sorted(list(set(time_diff.dropna())))

In [ ]:
time_diff.value_counts(dropna=False).sort_index()

In [ ]:
time_diff[time_diff > pd.Timedelta(minutes=10)]

### რაოდენობა, საზღვრები, მარტივი სტატისტიკა

In [ ]:
full_df.describe()

დამატებით 1-ლი და 99-ე მეასედების შემოწმება

In [ ]:
full_df.describe([0.01, 0.25, 0.5, 0.75, 0.99])

საშუალოსა და მედიანას შორის აბსოლუტური სხვაობა

In [ ]:
abs(full_df.describe().loc['mean'] - full_df.describe().loc['50%'])

დისკრეტული დიფერენციალის სტატისტიკა

In [ ]:
diff_df = full_df.diff()

In [ ]:
diff_df.describe([0.01, 0.25, 0.5, 0.75, 0.99])

### ვიზუალიზაცია

In [ ]:
f, ax = plt.subplots(figsize=(7, 4))

full_df.plot(ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(7, 4))

full_df['2015-01-20':'2015-01-22'].plot(ax=ax)

plt.tight_layout()
plt.show()

დღიური მაქსიმუმები

In [ ]:
daily_max = full_df.groupby(pd.Grouper(freq='1D')).max()

In [ ]:
daily_max_for_heatmap = daily_max.copy()

daily_max_for_heatmap.index = daily_max_for_heatmap.index.date

In [ ]:
import seaborn as sns
f, ax = plt.subplots(1, 1, figsize=(7, 3))
sns.heatmap(daily_max_for_heatmap.T, ax=ax)

for tick in ax.get_yticklabels():
    tick.set_rotation(0)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
plt.tight_layout()
plt.show()

#### ყოველთვიური საშუალო მაჩვენებლები

In [ ]:
monthly_means = full_df.groupby(pd.Grouper(freq='MS')).mean()

In [ ]:
monthly_means.index = monthly_means.index.date

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(9, 4))

monthly_means.plot(ax=ax)
ax.set_ylim(0, 70)
plt.legend(loc=4)
plt.tight_layout()
plt.show()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(9, 4))

monthly_means.plot.bar(ax=ax)
plt.legend(loc=4)
plt.tight_layout()
plt.show()

#### სიმჭიდროვის ვიზუალიზაცია

მონაცემების დღეებად დაყოფა და ზედდება

In [ ]:
daily_ALEQ = [group for group in full_df['A Leq'].groupby(pd.Grouper(freq='1D'))]
# full_df.columns

In [ ]:
for date, series in daily_ALEQ:
    series.name = date.date()
    series.index = series.index.time

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(7, 6))

for date, series in daily_ALEQ[:30]:
    if series.empty:
        continue
    series.plot(ax=ax[0], marker='o', lw=0, c='g')    
    series.plot(ax=ax[1], marker='o', lw=0, c='g', alpha=0.1)    
    
#     ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(7, 6))

for date, series in daily_ALEQ:
    if series.empty:
        continue
        
    if date.dayofweek == 2:
        series.plot(ax=ax[0], marker='o', alpha=0.1, lw=0, c='r')    
    if date.dayofweek == 6:
        series.plot(ax=ax[1], marker='o', alpha=0.1, lw=0, c='b')    
#     ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(7, 4))

for date, series in daily_ALEQ:
    if series.empty:
        continue
        
    if date.dayofweek == 2:
        series.plot(ax=ax, marker='o', alpha=0.1, lw=0, c='r')    
    if date.dayofweek == 6:
        series.plot(ax=ax, marker='o', alpha=0.1, lw=0, c='b')    
#     ax.legend()

plt.tight_layout()
plt.show()

#### საათობრივი განაწილება

In [ ]:
hourly_ALEQ = [group for group in full_df['A Leq'].groupby(pd.Grouper(freq='1H'))]

In [ ]:
group, series = hourly_ALEQ[0]

In [ ]:
hourly_ticks = pd.date_range(start='00:00:00', end='23:59:59', freq='1H').time

In [ ]:
def cut_hour(hourly_grouped, time):
    return pd.concat(
        [series
         for group, series in hourly_grouped 
         if group.time() == time]
    )

hourly_series_dict = {str(time): cut_hour(hourly_ALEQ, time) for time in hourly_ticks}


ამონარიდი ბოქს-გრაფიკის დოკუმენტაციიდან:

whis : float, sequence, or string (default = 1.5)

    As a float, determines the reach of the whiskers to the beyond the first and third quartiles. In other words, where IQR is the interquartile range (Q3-Q1), the upper whisker will extend to last datum less than Q3 + whis*IQR). Similarly, the lower whisker will extend to the first datum greater than Q1 - whis*IQR. Beyond the whiskers, data are considered outliers and are plotted as individual points. Set this to an unreasonably high value to force the whiskers to show the min and max values. Alternatively, set this to an ascending sequence of percentile (e.g., [5, 95]) to set the whiskers at specific percentiles of the data. Finally, whis can be the string 'range' to force the whiskers to the min and max of the data.


In [ ]:
# hourly_series_dict['00:00:00'].reset_index(drop=True)

In [ ]:
noindex_hourly_dict = {time: series.reset_index(drop=True) for time, series in hourly_series_dict.items()}

In [ ]:
noindex_hourly_df = pd.DataFrame(noindex_hourly_dict)

In [ ]:
f, ax = plt.subplots(figsize=(6, 4))

noindex_hourly_df.plot.box(ax=ax)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
    
plt.tight_layout()
plt.show()

### გამონაკლისები როგორც მნიშვნელოვანი სიდიდეები

In [ ]:
p999 = diff_df.describe([0.999]).loc['99.9%']

In [ ]:
p999

In [ ]:
significants = {col:full_df[diff_df[col] > p999[col]][col] 
                for col in diff_df}
    

In [ ]:
significants_df = pd.DataFrame(significants, columns=full_df.columns)

In [ ]:
significants_df

In [ ]:
f, ax = plt.subplots(figsize=(6, 4))

full_df['2015-01-12':'2015-01-22'].plot(alpha=0.7, ax=ax)
plt.gca().set_prop_cycle(None)
significants_df['2015-01-12':'2015-01-22'].plot(marker='^', lw=0, ax=ax)

plt.tight_layout()
plt.show()

#### დაგლუვება

In [ ]:
smooth_df = full_df.rolling(window=36).mean()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(6, 4))

full_df['2015-01-01':'2015-01-31'].plot(ax=ax, alpha=0.5)
smooth_df['2015-01-01':'2015-01-31'].plot(ax=ax, lw=3.0)

plt.tight_layout()
plt.show()

### კორელაციები

კორელაციის მატრიცა სრულ დროზე

In [ ]:
full_df.corr()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(6, 4))

cmap = sns.diverging_palette(100, 10, as_cmap=True)
mask = np.ones_like(full_df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = 0
# centered
sns.heatmap(full_df.corr(), center=0, square=True, cmap=cmap, mask=mask, linewidths=.5, cbar_kws={"shrink": .5})
# not centered
# sns.heatmap(full_df.corr(), square=True, cmap=cmap, mask=mask, linewidths=.5, cbar_kws={"shrink": .5})

plt.tight_layout()
plt.show()

დღიური კორელაცია

In [ ]:
daily_ALL = [group for group in full_df.groupby(pd.Grouper(freq='1D'))]

In [ ]:
corr_by_date = {date: daily_df.corr() for date, daily_df in daily_ALL}

In [ ]:
one_to_all_corr = {date: corr['A L95'] for date, corr in corr_by_date.items()}

In [ ]:
one_to_all_corr_df = pd.DataFrame(one_to_all_corr).T

In [ ]:
f, ax = plt.subplots(figsize=(6, 4))

one_to_all_corr_df.plot(ax=ax)

plt.tight_layout()
plt.show()

# შეჯამება

### შეგროვებულ მონაცემებში ნაპოვნი ხარვეზები:

 * მონაცემთა ჩაწერის სიხშირე არაა სტაბილურად 5 წუთი
 * მონაცემებს ჩაწერისას მუდმივად გამოტოვებულია შუაღამის მონაცემები
 * ჩანაწერთა უმრავლესობა დამრგვალებულია 5 წუთზე, მაგრამ არა ყველა
 * არის რამდენიმე ჩავარდნა, რომელიც 1 დღეზე მეტია. კვირების, თვეების და ა.შ. ქვეჯგუფების აჯამვისას შესაძლოა გამოიწვიოს უზუსტობა, რომელიც არ ექვემბდებარება გადამოწმებას
 * სიჩუმეში ხმაურის სტატისტიკური ხმაური მეტია, ამიტომ დღისა და ღამის ანალიზი ზოგიერთი ამოცანისთვის ერთმანეთისგან განცალკევებით სჯობს
 
### ანალიზის შესაძლო განვითარება


### მანქანური სწავლებისთვის მომზადება

 * მუდმივ სიდიდეთა სტატისტიკური ნორმალიზაცია
 * კატეგორიათა ბინარიზაცია
 * სტატისტიკური გამონაკლისების მოშორება
 * მონაცემთა კრებულის საწვრთნელ და შესამოწმებელ ნაწილად დაყოფა (Training  and Test data sets)
 